In [ ]:
!pip install fastparquet
import pandas as pd
import fastparquet
import numpy as np

# Чтение и загрузка данных

In [ ]:
# настройка ссылок для прочтения файла
path_currency = r'/content/historical_currency_exchange.parquet'
path_transactions = r'/content/transaction_fraud_data.parquet'

In [ ]:
# pip install fastparquet

In [ ]:
# Чтение файлов
currency_df = pd.read_parquet(path_currency)
transactions_df = pd.read_parquet(path_transactions, engine='fastparquet')

In [ ]:
# просмотр данных
transactions_df.head(5)

,transaction_id,customer_id,card_number,timestamp,vendor_category,vendor_type,vendor,amount,currency,country,...,ip_address,is_outside_home_country,is_high_risk_vendor,is_weekend,is_fraud,last_hour_activity.num_transactions,last_hour_activity.total_amount,last_hour_activity.unique_merchants,last_hour_activity.unique_countries,last_hour_activity.max_single_amount
0,TX_a0ad2a2a,CUST_72886,6646734767813109,2024-09-30 00:00:01.034820,Restaurant,fast_food,Taco Bell,294.87,GBP,UK,...,197.153.60.199,False,False,False,False,1197,3.349856e+07,105,12,1.925481e+06
1,TX_3599c101,CUST_70474,376800864692727,2024-09-30 00:00:01.764464,Entertainment,gaming,Steam,3368.97,BRL,Brazil,...,208.123.221.203,True,True,False,True,509,2.011476e+07,100,12,5.149117e+06
2,TX_a9461c6d,CUST_10715,5251909460951913,2024-09-30 00:00:02.273762,Grocery,physical,Whole Foods,102582.38,JPY,Japan,...,10.194.159.204,False,False,False,False,332,3.916385e+07,97,12,1.852242e+06
3,TX_7be21fc4,CUST_16193,376079286931183,2024-09-30 00:00:02.297466,Gas,major,Exxon,630.60,AUD,Australia,...,17.230.177.225,False,False,False,False,764,2.201260e+07,105,12,2.055798e+06
4,TX_150f490b,CUST_87572,6172948052178810,2024-09-30 00:00:02.544063,Healthcare,medical,Medical Center,724949.27,NGN,Nigeria,...,136.241.219.151,True,False,False,True,218,4.827636e+06,88,12,1.157231e+06


In [ ]:
transactions_df.shape

(7483766, 27)

In [ ]:
# просмотр данных
currency_df.head()

,date,AUD,BRL,CAD,EUR,GBP,JPY,MXN,NGN,RUB,SGD,USD
0,2024-09-30,1.443654,5.434649,1.351196,0.895591,0.747153,142.573268,19.694724,1668.736400,94.133735,1.280156,1
1,2024-10-01,1.442917,5.444170,1.352168,0.897557,0.746956,143.831429,19.667561,1670.694524,92.898519,1.284352,1
2,2024-10-02,1.449505,5.425444,1.348063,0.903056,0.752241,143.806861,19.606748,1669.653006,94.583198,1.286983,1
3,2024-10-03,1.456279,5.442044,1.351451,0.906018,0.754584,146.916773,19.457701,1670.097873,95.655442,1.294391,1
4,2024-10-04,1.460930,5.477788,1.355260,0.906452,0.761891,146.592323,19.363467,1649.763738,94.755337,1.296800,1


In [ ]:
currency_df.shape

(31, 12)

In [ ]:
transactions_df.isnull().sum()

,0
transaction_id,0
customer_id,0
card_number,0
timestamp,0
vendor_category,0
vendor_type,0
vendor,0
amount,0
currency,0
country,0


# Обработка прочитанных данных

In [ ]:
# Приводим дату транзакции и дату курсов валют к одному формату
transactions_df["date"] = transactions_df["timestamp"].dt.date
currency_df["date"] = pd.to_datetime(currency_df["date"]).dt.date

In [ ]:
transactions_df.columns

Для скорости на больших данных лучше использовать именно словарь, который релизует логику хеш-таблиц для быстрого доступа по ключу.

In [ ]:
# 1. Приводим дату транзакции к дате без времени
transactions_df['date_only'] = transactions_df['timestamp'].dt.floor('D')

# 2. Преобразуем currency_df в словарь {(date, currency): rate}
currency_dict = {
    (row.date, currency): row[currency]
    for _, row in currency_df.iterrows()
    for currency in currency_df.columns if currency != 'date'
}

# 3. Создаём Series с ключами для поиска в словаре
keys = pd.Series(list(zip(transactions_df['date_only'], transactions_df['currency'])))

# 4. Векторизованно применяем словарь с помощью map
rates = keys.map(currency_dict)

# 5. Пересчитываем amount в USD
transactions_df['amount_usd'] = transactions_df['amount'] / rates

# 6. Удаляем временный столбец
transactions_df.drop(columns='date_only', inplace=True)

In [ ]:
transactions_df[['amount','amount_usd']]

,amount,amount_usd
0,294.87,394.658011
1,3368.97,619.905757
2,102582.38,719.506409
3,630.60,436.808287
4,724949.27,434.430069
...,...,...
7483761,887.32,670.362079
7483762,356.06,385.402905
7483763,391.96,296.122166
7483764,601.71,601.710000


# Задания с тестирования

Задание 1. Доля мошеннических транзацкий.

In [ ]:
fraud_ratio = transactions_df['is_fraud'].mean()
print(f"Доля мошеннических транзакций: {fraud_ratio:.4f} ({fraud_ratio*100:.2f}%)")

Доля мошеннических транзакций: 0.1997 (19.97%)


Задание 2. В каких топ-5 странах зафиксировано больше всего мошеннических транзакций?

In [ ]:
fraud_counts = (
    transactions_df[transactions_df['is_fraud']]
    .groupby('country')
    .size()
    .sort_values(ascending=False)
)

print(fraud_counts.head(5))

country
Russia       299425
Mexico       298841
Brazil       298629
Nigeria      298600
Australia     37652
dtype: int64


Задание 3. Сколько транзакций в среднем совершает один клиент за час?

In [ ]:
transactions_df['hour'] = transactions_df['timestamp'].dt.floor('H')
avg = transactions_df.groupby(['customer_id', 'hour']).size().mean()


/tmp/ipython-input-4244079133.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  transactions_df['hour'] = transactions_df['timestamp'].dt.floor('H')


In [ ]:
print(avg)

2.4527761909397174


Задание 4. Какова доля мошенничества среди транзакций у продавцов с высоким риском (is_high_risk_vendor=True)?

In [ ]:
high_risk = transactions_df[transactions_df['is_high_risk_vendor'] == True]
fraud_ratio_high_risk = high_risk['is_fraud'].mean()
print(f"Доля мошенничества у продавцов с высоким риском: {fraud_ratio_high_risk:.4f} ({fraud_ratio_high_risk*100:.2f}%)")

Доля мошенничества у продавцов с высоким риском: 0.2000 (20.00%)


Задание 5. В каком городе наибольшая средняя сумма транзакций?

In [ ]:
avg_amount_by_city = transactions_df.groupby('city')['amount'].mean().sort_values(ascending=False)
print(avg_amount_by_city)


city
Unknown City    51315.780071
New York          568.872467
San Antonio       567.921894
San Jose          567.263760
Phoenix           565.730142
San Diego         565.593140
Chicago           565.277374
Houston           564.961388
Los Angeles       563.900769
Philadelphia      563.464431
Dallas            562.399533
Name: amount, dtype: float64


Задание 6. В каком городе выше всего средний чек по операциям, связанным с fast_food?

In [ ]:
top_city = (
    transactions_df.loc[transactions_df['vendor_type'] == 'fast_food']
    .groupby('city')['amount']
    .mean().sort_values(ascending=False)
)
print(top_city)

city
Unknown City    15214.407916
Chicago           264.450281
New York          263.296441
San Antonio       258.169691
Los Angeles       255.950324
San Diego         254.343884
Dallas            248.685716
San Jose          247.892502
Houston           246.913137
Philadelphia      243.440463
Phoenix           242.334715
Name: amount, dtype: float64


Задание 7. Каково среднее для всех немошеннических операций при пересчете в доллары США?

In [ ]:
transactions_df[transactions_df['is_fraud']==0]['amount_usd'].mean()

np.float64(459.7826108501768)

Задание 8. Каково среднеквадратичное отклонение среди всех немошеннических операций при пересчете в доллары США?

In [ ]:
transactions_df[transactions_df['is_fraud']==0]['amount_usd'].std()

417.0077334756002

Задание 9. Каково среднее для всех мошеннических операций при пересчете в доллары США?

In [ ]:
transactions_df[transactions_df['is_fraud']==1]['amount_usd'].mean()

np.float64(874.6067503080812)

Задание 10. Каково среднеквадратичное отклонение среди всех мошеннических операций при пересчете в доллары США?

In [ ]:
transactions_df[transactions_df['is_fraud']==1]['amount_usd'].std()

1349.8826922066885

Задание 11.


In [ ]:
# 1. Группируем по клиенту и считаем медиану уникальных продавцов
median_unique_merchants = transactions_df.groupby('customer_id')['last_hour_activity.unique_merchants'].median()

# 2. Находим 95-й перцентиль медиан
threshold = median_unique_merchants.quantile(0.95)

# 3. Считаем клиентов с медианой выше порога
count_risky_clients = (median_unique_merchants > threshold).sum()

print(f"Клиентов с потенциально опасным поведением: {count_risky_clients}")

Клиентов с потенциально опасным поведением: 229


Index(['transaction_id', 'customer_id', 'card_number', 'timestamp',
       'vendor_category', 'vendor_type', 'vendor', 'amount', 'currency',
       'country', 'city', 'city_size', 'card_type', 'is_card_present',
       'device', 'channel', 'device_fingerprint', 'ip_address',
       'is_outside_home_country', 'is_high_risk_vendor', 'is_weekend',
       'is_fraud', 'last_hour_activity.num_transactions',
       'last_hour_activity.total_amount',
       'last_hour_activity.unique_merchants',
       'last_hour_activity.unique_countries',
       'last_hour_activity.max_single_amount', 'date', 'amount_usd', 'hour'],
      dtype='object')